# Test 1

In the estimation process for testing the following data has been used.

- Aggregate new vends data from [udm].[Machine_use_event_transaction] with vending consumer and new consumer vends for the first 16 weeks of the yearc 2021 as comparison baseline circa ~ 13514 as400id data
- BDEX data containing circa ~ 256K emails with crica 3.5% match on consumers in the EDW and circa ~ 99.5% match on client as400id
- Between new vends and BDEX dataset The following are observed:
 - Red Group: 2158
 - Yellow Group: 57
 - Blue Group: 11355
- as400id profile data has been redacted and is being investigated for conformance issues

- Across the Red Group of circa ~2158 as400ids there are a total of circa ~172k potential new emails representing targeting opportunity
- As per business understanding redacting as400ids with target size less than 10 reduces target as400id to circa ~ 1778 as400ids and circa ~ 170187 emails
- Assuming a circa ~ 1/3rd  split of target and email conversion of ~2.6%, FB conversion of ~10% and email+FB conversion of  ~4.7%, leads to circa ~ 6213 new consumer vends
- The control versus required conversion rate is circa ~ 1.7% versus 2.1%- A required incrementality of 24.0% 

**Now below gives the number of weeks required to run the tes to ensure, x% power in a test/ Control that the study has a x% chance of ending up with a p value of less than 5%**

| % Strength | Weeks to run |
| -----------|--------------|
|80%         | ~ 1.5 Weeks  |
|90%         |       WIP       |
|95%         |   5.5 Weeks  |



In [1]:
import pandas as pd

In [2]:
 df=pd.read_csv('/content/new.txt', sep='\s+')

In [3]:
import pandas as pd
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
# mc= pd.read_csv('/gdrive/MyDrive/CE/machines.csv', names=['as400id', 'machine_num'])

In [ ]:
# af= pd.read_csv('/gdrive/MyDrive/CE/as400idprofile.csv', names= ['as400id', 'Region','Industry','IndustrySubType', 'LineOfBiz', 'AvgAge', 'PropertyClass', 'NumLaundryRooms'])

In [ ]:
# af.as400id= af.as400id.astype(str)

In [ ]:
# af= af.merge(mc, how='outer', on='as400id')

In [ ]:
# af.shape

In [ ]:
# len(set(mc.as400id.unique()).difference(set(af.as400id.unique())))

In [4]:
df.shape

(13538, 5)

In [5]:
ditch=[]
for a in df.as400id:
  try:
    int(a)
  except:
    ditch.append(a)

In [6]:
ditch

['7308204-6',
 '7312432-7',
 '7312478-1',
 '75063A',
 'ACCL123',
 'Dal0001',
 'eddie0330',
 'MIADEMO',
 'NJR2451',
 'NJR2452',
 'NYR0568',
 'NYR1682',
 'NYR2471',
 'nyr3757',
 'NYR9479',
 'ORLDEMO',
 'Test1111',
 'testDDD',
 'TTI1234',
 'ttitest02',
 'UM1072']

In [7]:
ditch.append('0')
ditch.append('000000000')
ditch.append('9999999')
df=df.loc[~df.as400id.isin(ditch)]

In [8]:
df.shape

(13514, 5)

In [ ]:
# df1=df.loc[df3.PropertyClass.isin(['Class B', 'Class C', 'Class D'])]

In [9]:
bd=pd.read_excel('/gdrive/MyDrive/CE/BDEX to UDM Data Mapping - May 13, 2021.xlsx')

In [10]:
bd['udm_user_found']=bd['UDM Consumer User ID'].map({'No Match Found': False}).fillna(True)

In [11]:
met_dict= {'email_address': 'count', 'udm_user_found': 'sum'}
bf= bd.loc[bd.Vendor=='DI'].groupby('Client Account AS400 ID').agg(met_dict).reset_index()
bf.columns=['as400id', 'emails', 'users_found']
bf['opportunity_size']= bf.emails- bf.users_found
bf['bdex']= 'Y'

In [12]:
len(set(df.as400id.unique()).difference(set(bf.as400id.unique())))

11355

In [13]:
len(set(bf.as400id.unique()).difference(set(df.as400id.unique())))

57

In [14]:
len(set(bf.as400id.unique()).intersection(set(df.as400id.unique())))

2158

In [15]:
len(df.as400id.unique())

13513

In [16]:
df1= df.merge(bf, how='left', on='as400id')

In [17]:
df1.bdex.fillna('N', inplace= True)

In [18]:
df1.bdex.value_counts()

N    11356
Y     2158
Name: bdex, dtype: int64

In [19]:
import numpy as np

In [20]:
df2= df1.loc[~(df1.opportunity_size.fillna(9999999999999999999) < 10)]

In [ ]:
# df1['applied_cutoff']= np.where(df1.opportunity_size < 10,0, df1.opportunity_size)
                                

In [21]:
df2.bdex.value_counts()

N    11356
Y     1778
Name: bdex, dtype: int64

In [22]:
df2.new_vends_pct= df2.new_vends/ df2.all_vends

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [23]:
df2.columns

Index(['as400id', 'uiq_users', 'all_vends', 'new_vends', 'other_vends',
       'emails', 'users_found', 'opportunity_size', 'bdex'],
      dtype='object')

In [24]:
met_dict1= {'as400id': 'nunique', 'uiq_users': 'sum', 'new_vends': 'sum', 'all_vends': 'sum','opportunity_size': 'sum' }

In [25]:
df3= df2.groupby('bdex').agg(met_dict1)

In [26]:
df3

,as400id,uiq_users,new_vends,all_vends,opportunity_size
bdex,,,,,
N,11355,618819,335784,14633289,0.0
Y,1778,73765,37734,2183926,170187.0


In [27]:
df3['new_vends_pct']= df3.new_vends/ df3.all_vends

In [28]:
df3

,as400id,uiq_users,new_vends,all_vends,opportunity_size,new_vends_pct
bdex,,,,,,
N,11355,618819,335784,14633289,0.0,0.022947
Y,1778,73765,37734,2183926,170187.0,0.017278


In [29]:
df3= df2.groupby('bdex').agg(met_dict1)

In [30]:
df3['new_vends_pct']= df3.new_vends/ df3.all_vends

In [31]:
df3

,as400id,uiq_users,new_vends,all_vends,opportunity_size,new_vends_pct
bdex,,,,,,
N,11355,618819,335784,14633289,0.0,0.022947
Y,1778,73765,37734,2183926,170187.0,0.017278


In [32]:
df31= df2.groupby('bdex')['new_vends'].agg('describe')

In [33]:
df31

,count,mean,std,min,25%,50%,75%,max
bdex,,,,,,,,
N,11356.0,29.568862,81.955195,0.0,3.0,10.0,27.0,2987.0
Y,1778.0,21.222722,42.336832,0.0,4.0,9.0,22.0,686.0


In [34]:
df21= df2.copy(deep=True)

In [35]:
df21.loc[(df2.bdex=='N') & (df21.new_vends.between(df31.loc['Y'].loc['mean']-.15*df31.loc['Y'].loc['std'], df31.loc['Y'].loc['mean']+.15*df31.loc['Y'].loc['std'])), 'bdex']= 'cntrl'

In [36]:
df32= df21.groupby('bdex')['new_vends'].agg('describe')

In [37]:
df32

,count,mean,std,min,25%,50%,75%,max
bdex,,,,,,,,
N,9722.0,31.193479,88.459337,0.0,3.0,7.0,32.0,2987.0
Y,1778.0,21.222722,42.336832,0.0,4.0,9.0,22.0,686.0
cntrl,1634.0,19.902693,3.663722,15.0,17.0,19.0,23.0,27.0


In [38]:
fb= email= efb= round(df3.loc['Y', 'opportunity_size']/3.0,0)

In [39]:
fb_cvr= 10/100
email_cvr= 2.7/100

In [40]:
import statistics as stat

In [ ]:
efb_cvr= stat.harmonic_mean([fb_cvr, email_cvr])

In [ ]:
new_vends= round(.4*fb*fb_cvr+ email*email_cvr+ efb*efb_cvr,0)

In [ ]:
new_vends

6213.0

In [ ]:
new_target_mean= (new_vends*10+df31.loc['Y'].loc['mean']*df31.loc['Y'].loc['count'])/df31.loc['Y'].loc['count']

In [ ]:
new_target_mean

56.166479190101235

In [ ]:
cntrl_mean= df32.loc['cntrl'].loc['mean']

In [ ]:
pop_std= df21.new_vends.std(ddof=1)

In [ ]:
pop_std

77.83319629544535

In [ ]:
es= np.abs((new_target_mean-cntrl_mean))/ pop_std

In [ ]:
n80= 2*((1.6+.84)/es)

In [ ]:
import numpy as np
from statsmodels.stats.power import TTestIndPower
import matplotlib.pyplot as plt

In [ ]:
prop= [1,1,1,1,1,1,1]
for alpha in [.025,.5,.1]:
  for i,week in enumerate([1,2,3,4,6,8,16]):
    nv= new_vends*prop[i]
    new_target_mean= (nv+df31.loc['Y'].loc['mean']*df31.loc['Y'].loc['count'])/df31.loc['Y'].loc['count']
    es= np.abs((new_target_mean-cntrl_mean))/ pop_std
    p_analysis = TTestIndPower()
    power = p_analysis.solve_power(effect_size=es, alpha=alpha, nobs1 = samples, ratio =1778/1634)
    print('Alpha:',alpha, 'Weeks:', week, 'Converted%:', 100*prop[i], 'Power:', 100*power)

Alpha: 0.025 Weeks: 1 Converted%: 100 Power: 4.007217832558452
Alpha: 0.025 Weeks: 2 Converted%: 100 Power: 4.007217832558452
Alpha: 0.025 Weeks: 3 Converted%: 100 Power: 4.007217832558452
Alpha: 0.025 Weeks: 4 Converted%: 100 Power: 4.007217832558452
Alpha: 0.025 Weeks: 6 Converted%: 100 Power: 4.007217832558452
Alpha: 0.025 Weeks: 8 Converted%: 100 Power: 4.007217832558452
Alpha: 0.025 Weeks: 16 Converted%: 100 Power: 4.007217832558452
Alpha: 0.5 Weeks: 1 Converted%: 100 Power: 54.17669239474265
Alpha: 0.5 Weeks: 2 Converted%: 100 Power: 54.17669239474265
Alpha: 0.5 Weeks: 3 Converted%: 100 Power: 54.17669239474265
Alpha: 0.5 Weeks: 4 Converted%: 100 Power: 54.17669239474265
Alpha: 0.5 Weeks: 6 Converted%: 100 Power: 54.17669239474265
Alpha: 0.5 Weeks: 8 Converted%: 100 Power: 54.17669239474265
Alpha: 0.5 Weeks: 16 Converted%: 100 Power: 54.17669239474265
Alpha: 0.1 Weeks: 1 Converted%: 100 Power: 13.41329575606551
Alpha: 0.1 Weeks: 2 Converted%: 100 Power: 13.41329575606551
Alpha: 0

In [ ]:
n80/

78.89365171195574

# Old Work

In [ ]:
df3.head()

,as400id,uiq_users,new_vends,all_vends,opportunity_size,new_vends_pct
bdex,,,,,,
N,11355,618819,335784,14633289,0.0,0.022947
Y,1778,73765,37734,2183926,170187.0,0.017278


In [ ]:
df2['pct_new_vends']= df2.new_vends/ df2.all_vends

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df3

,as400id,uiq_users,new_vends,all_vends,opportunity_size,new_vends_pct
bdex,,,,,,
N,11355,618819,335784,14633289,0.0,0.022947
Y,1778,73765,37734,2183926,170187.0,0.017278


In [ ]:
cntrl_rate= 0.017278

In [ ]:
incer= (new_rate- cntrl_rate)/ cntrl_rate

In [ ]:
incer

0.24414777313604882

In [ ]:
df2['actual_after_cutoff']= df2.applied_cutoff*0.4

In [ ]:
df2['actual_wo_cutoff']= df2.opportunity_size*0.4

In [ ]:
68074.8

In [ ]:
# df1.bdex.fillna('N', inplace= True)

In [ ]:
df3['new_vend_pct']= df3.new_vends/df3.all_vends

In [ ]:
df3['new_vend_as400id']= df3.new_vends/ df3.as400id

In [ ]:
old_rate= 0.018057

In [ ]:
incer

0.24414777313604882

In [ ]:
std= np.sqrt(old_rate*(1-old_rate))

In [ ]:
one_week_avg= 2183926/16

In [ ]:
N80= 16*2*(std/(old_rate*incer))**2

In [ ]:
N80/ one_week_avg

0.21387852869946733

In [ ]:
N95= 16*2*(std/(old_rate*incer))**2

In [ ]:
N95= 26*16*2*(std/(old_rate*incer))**2/one_week_avg

In [ ]:
N95

5.56084174618615

In [ ]:
stat.harmonic_mean([N95,N80])

11.119565408147016

In [ ]:
df1.new_vends_pct

0        1.000000
1        0.000000
2        0.023403
3        0.013233
4        0.000000
           ...   
13509    0.008824
13510    0.004202
13511    0.036364
13512    0.006820
13513    0.045714
Length: 13514, dtype: float64

In [ ]:
2/50

0.04